# This file is for SSA-LSTM model
- 0.9995

In [ ]:
for idx in range(3):    
    import os
    import pandas as pd
    import numpy as np
    import tensorflow as tf
    import IPython
    import IPython.display
    from IPython.display import clear_output
    import matplotlib.pyplot as plt
    import sys

    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Flatten, Dense, Dropout

    ## Read in my python class
    from Window_Generator_package import Window_Generator

    train_path = './Data/Parameter_tuning_0.9995/504/Train/'
    test_path = './Data/Parameter_tuning_0.9995/504/Test/'
    full_path = './Data/Grouped/'

    # stock_example = pd.read_csv(path + 'Data/Grouped/Consumer and Service Sectors/ALL.AX.csv')
    def read_stock(path, file_name):
        stock = pd.read_csv(path + file_name, header = [0])
        stock_index = pd.to_datetime(stock['Date'])
        stock.index = stock_index
        stock = stock.drop('Date', axis = 1)
        return stock
    group = ['Consumer and Service Sectors', 'Financial, Healthcare, Technology, and Utilities Sectors', 'Industrial and Infrastructure Sectors']
    index = [0, 1, 2]
    def read_all(the_path):
        All_df = []
        files = [f for f in os.listdir(the_path) if f.endswith('.csv')]

        for name in files:
            stock = read_stock(the_path + '/', name)
            All_df.append(stock)
        return All_df, files
    train_All_df_0, train_files = read_all(train_path + group[index[0]])
    test_All_df_0, test_files = read_all(test_path + group[index[0]])
    full_All_df_0, full_files = read_all(full_path + group[index[0]])
    example_train = train_All_df_0[0]
    example_test = test_All_df_0[0]
    example_actual = full_All_df_0[0][5:]

    train_series_1 = example_train.iloc[:, 0]
    test_series_1 = example_test.iloc[:, 0]


    MAX_EPOCHS = 30

    def compile_and_fit(model, X_train, Y_train, patience=5):
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')

        model.compile(loss=tf.keras.losses.MeanSquaredError(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=[tf.keras.metrics.MeanAbsoluteError()])

        history = model.fit(X_train, Y_train, epochs=MAX_EPOCHS, verbose = 0)
        return history


    def lstm_model(window_length):
        lstm = Sequential()
        lstm.add(LSTM(128, kernel_regularizer= 'l2'))
        lstm.add(Dropout(0.3))
        lstm.add(Dense(64, activation='relu'))
        lstm.add(Dense(1, activation = 'linear'))
        return lstm

    lstm = lstm_model(window_length = 5)

    def obtain_prediction(X_train, Y_train, X_test, Y_test, window_length = 5):
        # The input train and test are all 3D list
        predicted_train_list = []
        predicted_test_list = []
        train_MAE_list = []
        test_MAE_list = []
        for i in range(5):
            lstm = lstm_model(window_length)
            history = compile_and_fit(lstm, X_train = X_train, Y_train = Y_train)
            # IPython.display.clear_output()
            train_performance = lstm.evaluate(X_train, Y_train)[1]
            test_performance = lstm.evaluate(X_test, Y_test)[1]
            # print('Train:', str(train_performance))
            # print('Test:', str(test_performance))
            Y_predict_train = lstm.predict(X_train)
            Y_predict_train = Y_predict_train.reshape(len(Y_predict_train))
            predicted_train_list.append(Y_predict_train)

            Y_predict_test = lstm.predict(X_test)
            Y_predict_test = Y_predict_test.reshape(len(Y_predict_test))
            predicted_test_list.append(Y_predict_test)

            train_MAE_list.append(train_performance)
            test_MAE_list.append(test_performance)

        train_MAE = np.mean(train_MAE_list)
        test_MAE = np.mean(test_MAE_list)

        predicted_train_list = np.array(predicted_train_list)
        predicted_train_list = np.mean(predicted_train_list, axis = 0)
        predicted_test_list = np.array(predicted_test_list)
        predicted_test_list = np.mean(predicted_test_list, axis = 0)
        clear_output(wait=True)

        return predicted_train_list, predicted_test_list, train_MAE, test_MAE

    # Get result of 1 train and test series
    def obtain_ONE_series(train_series, test_series, window_length = 5, show = None):

        # Data Preparation
        w = Window_Generator(train_series = train_series, test_series = test_series)
        _, _, train_mean, train_std, test_mean, test_std = w.standardization(show=True)
        _, _, _, _ = w.window_generation(show = True)
        X_train, X_test, Y_train, Y_test = w.data_getter()
        train_denormalized = w.denormalize_train()
        test_denormalized = w.denormalize_test()

        # Training and Prediction
        predicted_train_list, predicted_test_list, train_MAE, test_MAE = obtain_prediction(X_train, Y_train, X_test, Y_test, window_length = window_length)
        # print('Train_MAE:', train_MAE)
        # print('Test_MAE:', test_MAE)

        # Prediction Generation (Are train_set)

        train_index = w.train_index[w.window_length:]
        test_index = w.test_index

        Y_predict_train = pd.Series(predicted_train_list, index = train_index)
        Y_predict_train = Y_predict_train * train_std[w.window_length:] + train_mean[w.window_length:]
        Y_actual_train = Y_train.reshape(len(Y_train)) * train_std[w.window_length:] + train_mean[w.window_length:]
        Y_predict_train = pd.DataFrame(Y_predict_train, index = train_index)
        Y_actual_train = pd.DataFrame(Y_actual_train, index = train_index)

        Y_predict_test = pd.Series(predicted_test_list, index = test_index)
        Y_predict_test = Y_predict_test * test_std + test_mean
        Y_actual_test = Y_test.reshape(len(Y_test)) * test_std + test_mean
        Y_predict_test = pd.DataFrame(Y_predict_test, index = test_index)
        Y_actual_test = pd.DataFrame(Y_actual_test, index = test_index)

        # if (show != None):
        #   plt.plot(Y_actual_train, label = 'Train Actual')
        #   plt.plot(Y_predict_train, label = 'Train predict')
        #   plt.plot(Y_actual_test, label = 'Test Actual')
        #   plt.plot(Y_predict_test, label = 'Test predict')
        #   plt.legend()
        #   plt.show()

        return Y_actual_train, Y_predict_train, Y_actual_test, Y_predict_test

    def obtain_ONE_stock(train, test, show = None):
        length = len(train.columns)
        actual_train = []
        predict_train = []
        actual_test = []
        predict_test = []
        for i in range(length):
            train_series = train.iloc[:, i]
            test_series = test.iloc[:, i]
            Series_actual_train, Series_predict_train, Series_actual_test, Series_predict_test = obtain_ONE_series(train_series, test_series, window_length = 5, show = show)
            actual_train.append(Series_actual_train)
            predict_train.append(Series_predict_train)
            actual_test.append(Series_actual_test)
            predict_test.append(Series_predict_test)

        final_actual_train = pd.concat(actual_train)
        final_actual_train = final_actual_train.groupby(final_actual_train.index).sum()

        final_predict_train = pd.concat(predict_train)
        final_predict_train = final_predict_train.groupby(final_predict_train.index).sum()

        final_actual_test = pd.concat(actual_test)
        final_actual_test = final_actual_test.groupby(final_actual_test.index).sum()

        final_predict_test = pd.concat(predict_test)
        final_predict_test = final_predict_test.groupby(final_predict_test.index).sum()
        # if (show != None):
        #     plt.plot(final_actual_train, label = 'Train Actual')
        #     plt.plot(final_predict_train, label = 'Train predict')
        #     plt.plot(final_actual_test, label = 'Test Actual')
        #     plt.plot(final_predict_test, label = 'Test predict')
        #     plt.legend()
        #     plt.show()

        return final_actual_train, final_predict_train, final_actual_test, final_predict_test


    final_actual_train, final_predict_train, final_actual_test, final_predict_test = obtain_ONE_stock(example_train, example_test, show = True)

    Y_result = pd.concat([pd.concat([final_actual_train, final_predict_train], axis = 1), pd.concat([final_actual_test, final_predict_test], axis = 1)], axis = 0)
    Y_result.columns = ['Smoothed_Actual', 'Predict']
    Y_result = pd.concat([example_actual, Y_result], axis = 1)

    store_path = './Data/Parameter_tuning_0.9995/504_Prediction/SSA-CNN/'
    def obtain_all(the_index, window_length = 5, show = None):
        group_name = group[index[the_index]]
        train_All_df, train_files = read_all(train_path + group_name)
        test_All_df, test_files = read_all(test_path + group_name)
        full_All_df, full_files = read_all(full_path + group_name)
        total_length = len(full_All_df)

        for i in range(total_length):
            stock_name = full_files[i]
            train_series = train_All_df[i]
            test_series = test_All_df[i]
            final_actual_train, final_predict_train, final_actual_test, final_predict_test = obtain_ONE_stock(train_series, test_series, show = show)

            actual = full_All_df[i][5:]
            Y_result = pd.concat([pd.concat([final_actual_train, final_predict_train], axis = 1), pd.concat([final_actual_test, final_predict_test], axis = 1)], axis = 0)
            Y_result.columns = ['Smoothed_Actual', 'Predict']
            Y_result = pd.concat([actual, Y_result], axis = 1)
            Y_result.to_csv(store_path + group_name + '/' + stock_name)
            clear_output(wait=True)

    obtain_all(idx, show = True)

# This file is for SSA-LSTM model
- 0.9997

In [ ]:
for idx in range(3):    
    import os
    import pandas as pd
    import numpy as np
    import tensorflow as tf
    import IPython
    import IPython.display
    from IPython.display import clear_output
    import matplotlib.pyplot as plt
    import sys

    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Flatten, Dense, Dropout

    ## Read in my python class
    from Window_Generator_package import Window_Generator

    train_path = './Data/Parameter_tuning_0.9997/504/Train/'
    test_path = './Data/Parameter_tuning_0.9997/504/Test/'
    full_path = './Data/Grouped/'

    # stock_example = pd.read_csv(path + 'Data/Grouped/Consumer and Service Sectors/ALL.AX.csv')
    def read_stock(path, file_name):
        stock = pd.read_csv(path + file_name, header = [0])
        stock_index = pd.to_datetime(stock['Date'])
        stock.index = stock_index
        stock = stock.drop('Date', axis = 1)
        return stock
    group = ['Consumer and Service Sectors', 'Financial, Healthcare, Technology, and Utilities Sectors', 'Industrial and Infrastructure Sectors']
    index = [0, 1, 2]
    def read_all(the_path):
        All_df = []
        files = [f for f in os.listdir(the_path) if f.endswith('.csv')]

        for name in files:
            stock = read_stock(the_path + '/', name)
            All_df.append(stock)
        return All_df, files
    train_All_df_0, train_files = read_all(train_path + group[index[0]])
    test_All_df_0, test_files = read_all(test_path + group[index[0]])
    full_All_df_0, full_files = read_all(full_path + group[index[0]])
    example_train = train_All_df_0[0]
    example_test = test_All_df_0[0]
    example_actual = full_All_df_0[0][5:]

    train_series_1 = example_train.iloc[:, 0]
    test_series_1 = example_test.iloc[:, 0]


    MAX_EPOCHS = 30

    def compile_and_fit(model, X_train, Y_train, patience=5):
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')

        model.compile(loss=tf.keras.losses.MeanSquaredError(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=[tf.keras.metrics.MeanAbsoluteError()])

        history = model.fit(X_train, Y_train, epochs=MAX_EPOCHS, verbose = 0)
        return history


    def lstm_model(window_length):
        lstm = Sequential()
        lstm.add(LSTM(128, kernel_regularizer= 'l2'))
        lstm.add(Dropout(0.3))
        lstm.add(Dense(64, activation='relu'))
        lstm.add(Dense(1, activation = 'linear'))
        return lstm

    lstm = lstm_model(window_length = 5)

    def obtain_prediction(X_train, Y_train, X_test, Y_test, window_length = 5):
        # The input train and test are all 3D list
        predicted_train_list = []
        predicted_test_list = []
        train_MAE_list = []
        test_MAE_list = []
        for i in range(5):
            lstm = lstm_model(window_length)
            history = compile_and_fit(lstm, X_train = X_train, Y_train = Y_train)
            # IPython.display.clear_output()
            train_performance = lstm.evaluate(X_train, Y_train)[1]
            test_performance = lstm.evaluate(X_test, Y_test)[1]
            # print('Train:', str(train_performance))
            # print('Test:', str(test_performance))
            Y_predict_train = lstm.predict(X_train)
            Y_predict_train = Y_predict_train.reshape(len(Y_predict_train))
            predicted_train_list.append(Y_predict_train)

            Y_predict_test = lstm.predict(X_test)
            Y_predict_test = Y_predict_test.reshape(len(Y_predict_test))
            predicted_test_list.append(Y_predict_test)

            train_MAE_list.append(train_performance)
            test_MAE_list.append(test_performance)

        train_MAE = np.mean(train_MAE_list)
        test_MAE = np.mean(test_MAE_list)

        predicted_train_list = np.array(predicted_train_list)
        predicted_train_list = np.mean(predicted_train_list, axis = 0)
        predicted_test_list = np.array(predicted_test_list)
        predicted_test_list = np.mean(predicted_test_list, axis = 0)
        clear_output(wait=True)

        return predicted_train_list, predicted_test_list, train_MAE, test_MAE

    # Get result of 1 train and test series
    def obtain_ONE_series(train_series, test_series, window_length = 5, show = None):

        # Data Preparation
        w = Window_Generator(train_series = train_series, test_series = test_series)
        _, _, train_mean, train_std, test_mean, test_std = w.standardization(show=True)
        _, _, _, _ = w.window_generation(show = True)
        X_train, X_test, Y_train, Y_test = w.data_getter()
        train_denormalized = w.denormalize_train()
        test_denormalized = w.denormalize_test()

        # Training and Prediction
        predicted_train_list, predicted_test_list, train_MAE, test_MAE = obtain_prediction(X_train, Y_train, X_test, Y_test, window_length = window_length)
        # print('Train_MAE:', train_MAE)
        # print('Test_MAE:', test_MAE)

        # Prediction Generation (Are train_set)

        train_index = w.train_index[w.window_length:]
        test_index = w.test_index

        Y_predict_train = pd.Series(predicted_train_list, index = train_index)
        Y_predict_train = Y_predict_train * train_std[w.window_length:] + train_mean[w.window_length:]
        Y_actual_train = Y_train.reshape(len(Y_train)) * train_std[w.window_length:] + train_mean[w.window_length:]
        Y_predict_train = pd.DataFrame(Y_predict_train, index = train_index)
        Y_actual_train = pd.DataFrame(Y_actual_train, index = train_index)

        Y_predict_test = pd.Series(predicted_test_list, index = test_index)
        Y_predict_test = Y_predict_test * test_std + test_mean
        Y_actual_test = Y_test.reshape(len(Y_test)) * test_std + test_mean
        Y_predict_test = pd.DataFrame(Y_predict_test, index = test_index)
        Y_actual_test = pd.DataFrame(Y_actual_test, index = test_index)

        # if (show != None):
        #   plt.plot(Y_actual_train, label = 'Train Actual')
        #   plt.plot(Y_predict_train, label = 'Train predict')
        #   plt.plot(Y_actual_test, label = 'Test Actual')
        #   plt.plot(Y_predict_test, label = 'Test predict')
        #   plt.legend()
        #   plt.show()

        return Y_actual_train, Y_predict_train, Y_actual_test, Y_predict_test

    def obtain_ONE_stock(train, test, show = None):
        length = len(train.columns)
        actual_train = []
        predict_train = []
        actual_test = []
        predict_test = []
        for i in range(length):
            train_series = train.iloc[:, i]
            test_series = test.iloc[:, i]
            Series_actual_train, Series_predict_train, Series_actual_test, Series_predict_test = obtain_ONE_series(train_series, test_series, window_length = 5, show = show)
            actual_train.append(Series_actual_train)
            predict_train.append(Series_predict_train)
            actual_test.append(Series_actual_test)
            predict_test.append(Series_predict_test)

        final_actual_train = pd.concat(actual_train)
        final_actual_train = final_actual_train.groupby(final_actual_train.index).sum()

        final_predict_train = pd.concat(predict_train)
        final_predict_train = final_predict_train.groupby(final_predict_train.index).sum()

        final_actual_test = pd.concat(actual_test)
        final_actual_test = final_actual_test.groupby(final_actual_test.index).sum()

        final_predict_test = pd.concat(predict_test)
        final_predict_test = final_predict_test.groupby(final_predict_test.index).sum()
        # if (show != None):
        #     plt.plot(final_actual_train, label = 'Train Actual')
        #     plt.plot(final_predict_train, label = 'Train predict')
        #     plt.plot(final_actual_test, label = 'Test Actual')
        #     plt.plot(final_predict_test, label = 'Test predict')
        #     plt.legend()
        #     plt.show()

        return final_actual_train, final_predict_train, final_actual_test, final_predict_test


    final_actual_train, final_predict_train, final_actual_test, final_predict_test = obtain_ONE_stock(example_train, example_test, show = True)

    Y_result = pd.concat([pd.concat([final_actual_train, final_predict_train], axis = 1), pd.concat([final_actual_test, final_predict_test], axis = 1)], axis = 0)
    Y_result.columns = ['Smoothed_Actual', 'Predict']
    Y_result = pd.concat([example_actual, Y_result], axis = 1)

    store_path = './Data/Parameter_tuning_0.9997/504_Prediction/SSA-CNN/'
    def obtain_all(the_index, window_length = 5, show = None):
        group_name = group[index[the_index]]
        train_All_df, train_files = read_all(train_path + group_name)
        test_All_df, test_files = read_all(test_path + group_name)
        full_All_df, full_files = read_all(full_path + group_name)
        total_length = len(full_All_df)

        for i in range(total_length):
            stock_name = full_files[i]
            train_series = train_All_df[i]
            test_series = test_All_df[i]
            final_actual_train, final_predict_train, final_actual_test, final_predict_test = obtain_ONE_stock(train_series, test_series, show = show)

            actual = full_All_df[i][5:]
            Y_result = pd.concat([pd.concat([final_actual_train, final_predict_train], axis = 1), pd.concat([final_actual_test, final_predict_test], axis = 1)], axis = 0)
            Y_result.columns = ['Smoothed_Actual', 'Predict']
            Y_result = pd.concat([actual, Y_result], axis = 1)
            Y_result.to_csv(store_path + group_name + '/' + stock_name)
            clear_output(wait=True)

    obtain_all(idx, show = True)